In [1]:
!pip install pandas
!pip install selenium


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

In [86]:
df_link_data = pd.read_csv('data/links/link_data.csv')
df_link_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18106 entries, 0 to 18105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   link    18106 non-null  object
dtypes: object(1)
memory usage: 141.6+ KB


In [94]:
LIST_LINK_PRODUCT = df_link_data['link'].values.tolist()
print(len(LIST_LINK_PRODUCT))

18106


In [112]:
import re

def extract_type_estate_and_prId(url):
    # Define regular expressions for matching prId
    prId_pattern = r"pr(\d+)"

    # Match prId using regular expression
    prId_match = re.search(prId_pattern, url)

    # Extract prId if a match is found
    prId = prId_match.group(1) if prId_match else None

    # Extract type_estate based on specific strings in the URL
    type_estate_keywords = {
        'ban-nha-rieng': 'Nhà riêng',
        'ban-nha-mat-pho': 'Nhà mặt phố',
        'ban-nha-biet-thu-lien-ke': 'Nhà biệt thự liền kề',
        'ban-shophouse-nha-pho-thuong-mai': 'Shophouse/Nhà phố thương mại'
    }

    type_estate = 'Unknown'
    for keyword, type_name in type_estate_keywords.items():
        if keyword in url:
            type_estate = type_name
            break

    return type_estate, prId


[('Nhà riêng', '39511518'), ('Nhà riêng', '39250524'), ('Nhà biệt thự liền kề', '35464584'), ('Nhà mặt phố', '39813780'), ('Nhà biệt thự liền kề', '39815386'), ('Nhà mặt phố', '39857717'), ('Nhà riêng', '39718266'), ('Nhà biệt thự liền kề', '38137432'), ('Nhà biệt thự liền kề', '35991254'), ('Nhà riêng', '39813396'), ('Shophouse/Nhà phố thương mại', '39781734'), ('Nhà mặt phố', '39810164'), ('Nhà biệt thự liền kề', '38657225')]


In [118]:
LIST_PRODUCT = []

# Define a function to extract property details
def extract_property_details(url):
    property_details = {
        'pr_id': '',
        'type_estate': '',
        'district': '',
        'posted_date': '',
        'area': '',
        'price': '',
        'legal_document': '',
        'interior': '',
        'num_bedrooms': '',
        'num_bathrooms': '',
        'num_floors': '',
        'house_orientation': '',
        'balcony_orientation': '',
        'entrance': '',
        'frontage': ''
    }

    property_details['type_estate'], property_details['pr_id'] = extract_type_estate_and_prId(url)

    try:
        # Set up Selenium WebDriver
        with webdriver.Chrome() as driver:
            driver.get(url)
            
            # Get the page source
            page_source = driver.page_source

        soup = BeautifulSoup(page_source, "html.parser")

        try:
            # Find the breadcrumb element
            breadcrumb = soup.find('div', class_='re__breadcrumb')
            if breadcrumb:
                level_3_link = breadcrumb.find('a', {'level': '3'})
                if level_3_link:
                    property_details['district'] = level_3_link.text
            else:
                print(f"Breadcrumb not found for URL: {url}")
        except AttributeError as e:
            print(f"Error extracting district: {e}")

        # Extract posted date
        try:
            posted_date_span = soup.find('span', string='Ngày đăng')
            if posted_date_span:
                property_details['posted_date'] = posted_date_span.find_next_sibling('span').text.strip()
        except AttributeError as e:
            print(f"Error extracting posted_date: {e}")

        # Extract property specifications
        try:
            specs_content = soup.find_all('div', class_='re__pr-specs-content-item')
            # Mapping of titles to corresponding keys in property_details
            title_map = {
                'Diện tích': 'area',
                'Mức giá': 'price',
                'Pháp lý': 'legal_document',
                'Nội thất': 'interior',
                'Số phòng ngủ': 'num_bedrooms',
                'Số toilet': 'num_bathrooms',
                'Số tầng': 'num_floors',
                'Hướng nhà': 'house_orientation',
                'Hướng ban công': 'balcony_orientation',
                'Đường vào': 'entrance',
                'Mặt tiền': 'frontage'
            }

            # Extract and map property details
            for item in specs_content:
                title = item.find(class_='re__pr-specs-content-item-title').text.strip()
                if title in title_map:
                    property_details[title_map[title]] = item.find(class_='re__pr-specs-content-item-value').text.strip()
        except AttributeError as e:
            print(f"Error extracting property details: {e}")
        
    except Exception as e:
        print(f"Failed to extract data from {url}: {e}")

    return property_details

# Function to write property details to CSV
def write_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, mode='a', header=not pd.io.common.file_exists(filename))

# Function to get existing pr_id from CSV
def get_existing_pr_ids(filename):
    if pd.io.common.file_exists(filename):
        df = pd.read_csv(filename)
        return set(df['pr_id'])
    return set()

def scrape_property_details(urls, filename):
    existing_pr_ids = get_existing_pr_ids(filename)
    scraped_properties = []
    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(extract_property_details, url): url for url in urls}
        try:
            for future in as_completed(futures):
                url = futures[future]
                try:
                    result = future.result()
                    if result['pr_id'] not in existing_pr_ids:
                        scraped_properties.append(result)
                        write_to_csv([result], filename)
                        existing_pr_ids.add(result['pr_id'])
                except Exception as e:
                    print(f"Failed to extract data from {url}: {e}")
        except KeyboardInterrupt:
            for f in futures:
                f.cancel()
            raise

    return scraped_properties

LIST_PRODUCT = scrape_property_details(LIST_LINK_PRODUCT, 'data/raw_data.csv')
print(LIST_PRODUCT)

Breadcrumb not found for URL: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-nghiem-xuan-yem-phuong-dai-kim-prj-the-manor-central-park/-lien-ke-vua-o-vua-kinh-doanh-mat-oto-tranh-rong-30m2-pr39847124
Breadcrumb not found for URL: https://batdongsan.com.vn/ban-nha-rieng-duong-thanh-binh-phuong-mo-lao/ban-la-khe-yen-nghia-phu-luong-ha-dong-hn-pr39844629
Breadcrumb not found for URL: https://batdongsan.com.vn/ban-shophouse-nha-pho-thuong-mai-duong-nguyen-xien-phuong-dai-kim-prj-the-manor-central-park/chinh-chu-can-ban-mat-di-bo-gia-re-nhat-thi-truong-canh-truong-quoc-te-dwight-pr39748332


KeyboardInterrupt: 

In [6]:
print(len(LIST_PRODUCT))

1000


In [9]:
df_link_product = pd.DataFrame(LIST_PRODUCT)
df_link_product.to_csv('data/raw_data.csv', index=False)

In [10]:
# Read the CSV file into a pandas DataFrame
df = pd.read_csv("data/raw_data.csv")
df1 = pd.read_csv("data/raw_data_2.csv")
df2 = pd.read_csv("data/raw_data_1.csv")

# Concatenate the DataFrames
mf = pd.concat([df, df1, df2], ignore_index=True)

# Write the merged file to a new CSV
mf.to_csv('data/raw_data_real.csv')

In [45]:
df = pd.read_csv('data/raw_data_real.csv')
df.info()
# df.to_csv('data/raw_data_real.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           10000 non-null  int64 
 1   type_estate          9091 non-null   object
 2   district             9091 non-null   object
 3   price_per_sqm        8093 non-null   object
 4   posted_date          9082 non-null   object
 5   area                 9082 non-null   object
 6   price                9082 non-null   object
 7   legal_document       7914 non-null   object
 8   interior             5510 non-null   object
 9   num_bedrooms         6358 non-null   object
 10  num_bathrooms        5706 non-null   object
 11  num_floors           8014 non-null   object
 12  house_orientation    1947 non-null   object
 13  balcony_orientation  1230 non-null   object
 14  entrance             4401 non-null   object
 15  frontage             6222 non-null   object
dtypes: in

In [81]:
df = pd.read_csv('data/raw_data_real.csv')
df.drop(['house_orientation', 'balcony_orientation'], axis=1, inplace=True)
# # Drop rows with empty data
# df.dropna(how='all', inplace=True)

df.to_csv('data/raw_data_test.csv', index=False)
# df.info()

In [52]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 860.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 6.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [82]:
df = pd.read_csv('data/raw_data_test.csv')
# df[['num_floors']].to_csv('data/num_floors.csv', index=False)

In [83]:

# Display the original DataFrame
print("Original DataFrame:")
print(df)

# Step 1: Remove rows with entirely empty critical fields
critical_fields = ['type_estate', 'district', 'posted_date', 'area', 'price']
df.dropna(subset=critical_fields, how='all', inplace=True)

# Remove specific substrings and convert to appropriate types
df['num_bedrooms'] = df['num_bedrooms'].str.replace(' phòng', '').astype(float)
df['num_bathrooms'] = df['num_bathrooms'].str.replace(' phòng', '').astype(float)
df['num_floors'] = df['num_floors'].str.replace(' tầng', '').astype(float)
df['entrance'] = df['entrance'].str.replace(' m', '').str.replace(',', '.').astype(float)
df['frontage'] = df['frontage'].str.replace(' m', '').str.replace(',', '.').astype(float)
df['area'] = df['area'].str.replace(' m²', '').str.replace(',', '.').astype(float)

# Step 3: Fill missing numerical values with the mean of their respective columns
df['num_bedrooms'] = df['num_bedrooms'].fillna(df['num_bedrooms'].mean())
df['num_bathrooms'] = df['num_bathrooms'].fillna(df['num_bathrooms'].mean())
df['num_floors'] = df['num_floors'].fillna(df['num_floors'].mean())
df['entrance'] = df['entrance'].fillna(df['entrance'].mean())
df['frontage'] = df['frontage'].fillna(df['frontage'].mean())
df['area'] = df['area'].fillna(df['area'].mean())

# Step 4: Fill missing categorical values with 'Not provided'
df.fillna({
    'legal_document': 'Not provided',
    'interior': 'Not provided'
}, inplace=True)

# Step 5: Standardize data types for cleaned numerical fields
df['num_bedrooms'] = df['num_bedrooms'].astype(int)
df['num_bathrooms'] = df['num_bathrooms'].astype(int)
df['num_floors'] = df['num_floors'].astype(int)
df['entrance'] = df['entrance'].astype(float)
df['frontage'] = df['frontage'].astype(float)

# Step 6: Normalize and encode categorical strings
df['type_estate'] = df['type_estate'].str.strip()
df['district'] = df['district'].str.strip()
df['legal_document'] = df['legal_document'].str.strip()
df['interior'] = df['interior'].str.strip()

# Convert categorical fields to numerical codes if needed
label_encoders = {}
categorical_fields = ['type_estate', 'district', 'legal_document', 'interior']

for field in categorical_fields:
    le = LabelEncoder()
    df[field] = le.fit_transform(df[field])
    label_encoders[field] = le  # Store the label encoder if you need to inverse transform later

# Display the cleaned and encoded DataFrame
print("Cleaned and Encoded DataFrame:")
print(df)

# Optionally save the cleaned data to a new CSV file
df.to_csv('cleaned_data.csv', index=False)

Original DataFrame:
      Unnamed: 0                    type_estate      district  \
0              0          Nhà biệt thự, liền kề       Hà Đông   
1              1  Shophouse, nhà phố thương mại       Hà Đông   
2              2                            NaN           NaN   
3              3                    Nhà mặt phố     Hoàng Mai   
4              4          Nhà biệt thự, liền kề     Long Biên   
...          ...                            ...           ...   
9995        9995                      Nhà riêng   Nam Từ Liêm   
9996        9996                    Nhà mặt phố  Hai Bà Trưng   
9997        9997                            NaN           NaN   
9998        9998                    Nhà mặt phố  Hai Bà Trưng   
9999        9999                    Nhà mặt phố     Hoàn Kiếm   

         price_per_sqm posted_date      area       price  legal_document  \
0     ~173,12 triệu/m²  13/05/2024     80 m²    13,85 tỷ  Sổ đỏ/ Sổ hồng   
1       ~36,8 triệu/m²  11/05/2024    625 m²   